In [1]:
import socket
import os
import uuid
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
AUDIO_FOLDER = f'.{os.sep}'

# Load models
model_prompt = MusicGen.get_pretrained("facebook/musicgen-stereo-small", device='cpu')
# model_melody = MusicGen.get_pretrained("facebook/musicgen-stereo-melody", device='cpu')
model_melody = None
print('Done loading models')

Done loading models


/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [4]:
# Replace this function with your actual audio generation logic
def generate_audio(prompt, temperature, topk, topp, cfg, samples, duration, dropped=False):
    audio_file_paths = [os.path.join(AUDIO_FOLDER, f"{uuid.uuid4()}") for _ in range(samples)]
    
    if dropped:
        melody, sr = torchaudio.load(dropped)
        model_melody.set_generation_params(
            duration=duration,
            temperature=temperature,
            top_k=topk,
            top_p=topp,
            cfg_coef=cfg
        )
        for audio_file_path in audio_file_paths:
            wav = model_melody.generate_with_chroma([prompt], melody[None].expand(1, -1, -1), sr)
            audio_write(audio_file_path, wav[0].cpu(), model_melody.sample_rate, strategy="loudness")
    else:
        model_prompt.set_generation_params(
            duration=duration,
            temperature=temperature,
            top_k=topk,
            top_p=topp,
            cfg_coef=cfg
        )
        for audio_file_path in audio_file_paths:
            wav = model_prompt.generate([prompt])
            audio_write(audio_file_path, wav[0].cpu(), model_prompt.sample_rate, strategy="loudness")

    return [f"{audio_file_path}.wav" for audio_file_path in audio_file_paths]

In [5]:
def start_socket_server(host='127.0.0.1', port=65432):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)
    print(f"Server listening on {host}:{port}...")

    while True:
        conn, addr = server_socket.accept()
        print(f"Connected by {addr}")

        # Receive the request from the client
        data = conn.recv(4096).decode('utf-8')
        if not data:
            break
        print(f"Received request: {data}")

        # Expecting the data to be in the format: 'prompt|temperature|topk|topp|cfg|samples|duration'
        params = data.split('|')
        if len(params) == 7:
            prompt, temperature, topk, topp, cfg, samples, duration = params
            temperature = float(temperature)
            topk = int(topk)
            topp = float(topp)
            cfg = int(cfg)
            samples = int(samples)
            duration = int(duration)

            # Generate audio
            audio_file_paths = generate_audio(prompt, temperature, topk, topp, cfg, samples, duration)
            audio_filenames = [os.path.basename(audio_file_path) for audio_file_path in audio_file_paths]
            response = '|'.join(audio_filenames)

            # Send response back to client
            conn.sendall(response.encode('utf-8'))
        else:
            conn.sendall(b'Invalid request format')
        
        # Close the connection
        conn.close()

if __name__ == "__main__":
    start_socket_server()

Server listening on 127.0.0.1:65432...
Connected by ('127.0.0.1', 40946)
Received request: test_prompt|0.700000|250|0.950000|2|3|15


CLIPPING ./f267bd6b-d412-4fc6-9c84-aaf6e8aa9fc0 happening with proba (a bit of clipping is okay): 0.0002208333316957578 maximum scale:  2.6917386054992676


FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'